# """
    THIS SCRIPT PERFORM PHOSCPREDICTION FROM IMAGE CROPS
    
    1. THE IMAGE CROPS CAN BE FROM A YOLOV3 PREDICTION SCRIPT 
    2. THE IMAGE CROPS CAN BE FROM A ORIGINAL BB FROM LOCALIZATION DATA
    3. THIS SCRIPT PROJECT BACK THE RESULT ON ORIGINAL IMAGE WHERE WE
    CAN SEE CORRECT AND WRONG PREDICTION.
    4. IMAGE CROPS HAS SPECIFIC NAME FORMAT
    
"""

In [1]:
import tensorflow as tf
tf.__version__

2022-04-17 18:45:17.390650: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-17 18:45:17.390667: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


'2.8.0'

"""
This is PHOSCNET model prediction script
"""

In [2]:
from phoscPipeline import * # contains phosc model 
from phos_label_generator import gen_label
from phoc_label_generator import gen_phoc_label
import os

In [3]:
model=build_model()
MODEL="/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/new_IAM_16_"
model.load_weights(MODEL+".h5")
#model=tf.keras.models.load_model(MODEL+".h5")
#print(MODEL)

2022-04-17 18:45:25.598902: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-17 18:45:25.599192: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-17 18:45:25.599238: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-04-17 18:45:25.599275: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-04-17 18:45:25.611621: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [4]:
"""
    returns a dictionary having word as a key and PHOSC representation as a value 
"""

def get_comb_label(x):
    
    phos_labels=gen_label(x)
    phoc_labels=gen_phoc_label(x)
    test_labels=dict()
    
    for x in phos_labels:
        test_labels[x]=np.concatenate((phos_labels[x],phoc_labels[x]),axis=0)
    
    return test_labels

#train_word_label=get_comb_label(list(set(df_lex['Word'])))

def similarity(x,y):
    return 1000*np.dot(x,y)/(LA.norm(x)*LA.norm(y))


"""

    below predict word from  image crop  image  using PHOSCNET 
    
"""

In [35]:
def predict(path2Image,test_word_label):
    
    x=img_to_array(load_img(path2Image))

    x = np.expand_dims(x, axis=0)
    y_pred=model.predict(x)
    y_pred=np.squeeze(np.concatenate((y_pred[0],y_pred[1]),axis=1))
    #print(y_pred)
    mx=0
    
    for k in test_word_label:
        temp=similarity(y_pred,test_word_label[k])
        if temp>mx:
            mx=temp
            op=k
            
    return op,mx


In [6]:
"""
    gather all words corresponding to specific single image

"""

import re,os

#images=['c04-139.png', 'g06-018r.png', 'a01-072x.png', 'h01-010.png', 'c04-050.png', 'm01-032.png', 'g06-042r.png', 'g06-026i.png', 'h07-080.png', 'e04-022.png','g06-047h.png']

images=["m01-049.png","m01-084.png",'m01-090.png', 'm01-110.png', 'm01-049.png', 'm01-060.png', 'm01-104.png', 'm01-095.png', 'm01-121.png', 'm01-115.png', 'm01-084.png', 'm01-079.png']

orgImage=images[0]

testCropPath="./data/crop2/" #"./data/dataset/forms//" #

#testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/IAM_Data/"


def getPhoscFromText(testCropPath,orgImage):
    
    """
        input:image directory and image name containing image crops,  Image crops are expected
        to have name with specific convention
        
        output: dictionary having word as key and PHOSC representation as value
        
        working: input contains paths to dictionary containing all crops of words present in image
        from labels of image in folder, original word labels are extracted and from it PHOSCNET representation 
        is derived
        
    """
    
    regex = re.compile('[^a-zA-Z]')
    allWords=[]


    for indx,imgName in enumerate(os.listdir(testCropPath+orgImage)):

        word=imgName.split("]_")[1]
        word=word.split("_.png")[0]
        word= regex.sub('', word)
        allWords.append(word)

    allWords=list(set(allWords))
    test_word_label=get_comb_label(allWords)
    #print(len(allWords))
    #allWords
    return test_word_label,allWords
    
    
#test_word_label,allWords=getPhoscFromText(testCropPath,orgImage)
#len(allWords)


def resizeAll(resizeFolder):
    
    expCount=0
    
    for orgImage in os.listdir(resizeFolder):            
        
        tempFolder=os.path.join(resizeFolder,orgImage)
        
        if not os.path.isdir(tempFolder):
            continue

        for cropName in os.listdir(tempFolder):
            try:
                tempImgPath=os.path.join(tempFolder,cropName)

                img=cv2.imread(tempImgPath)
                #print(img.shape)
                img=cv2.resize(img,(250,50))
                cv2.imwrite(tempImgPath,img)
                
            except Exception as e:
                expCount+=1
                pass
            
    print("expCount:",expCount)    

#resizeAll(testCropPath)


expCount: 14187


In [10]:
"""
    DONT USE!!!

    below part takes input single crop folder name and image name as a input
    and perform PHOSC prediction on it, It also plots the prediction on
    original image
"""

import cv2
import os

regex = re.compile('[^a-zA-Z]')

testImgPath=".//data/dataset/forms//" 
testCropPath="./data/crops//"+orgImage+"//"
savePath="/data//cropPHOSCVisual2//"

print(" orgImage:",orgImage)

if not os.path.isfile(testImgPath+orgImage):
    print(" File not present!!!")
else:
    print(" File found!!",testImgPath+orgImage)

    image=cv2.imread(testImgPath+orgImage)
    print(" original image name:",orgImage," original image shape:",image.shape)

matchCount=0
missMatch=0

for indx,cropName in enumerate(os.listdir(testCropPath)):
    
    print("")
    print("cropName:",cropName)
    word=cropName.split("]_")[1]
    word=word.split("_.png")[0]
    word= regex.sub('', word)
    
    if len(word)<2: # skip 
        continue
    
    cor=cropName.split("[")[1]
    cor=cor.split("]")[0]
    cor="["+cor+"]"
    cor=eval(cor)
    
    x1,y1,x2,y2=int(cor[0]),int(cor[1]),int(cor[2]),int(cor[3])
        
    #imgName=imgName.split("_")[0]
    
    predWord,score=predict(testCropPath+cropName)
    
    #print(" word:",word," ",predWord," score:",score," coordinate:",x1,y1,x2,y2)
    
    if word==predWord:
        #cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,255,0), 3)
        #cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        matchCount+=1
    else:
        
        cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,0,255), 3)
        cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        missMatch+=1
    

cv2.putText(image,str([matchCount,missMatch,matchCount/(matchCount+missMatch)]), (15,100), cv2.FONT_HERSHEY_SIMPLEX, 3, 255)
cv2.imwrite(savePath+orgImage,image)

print("match",matchCount)
print("miss match:",missMatch)
print(" Accuracy:",matchCount/(matchCount+missMatch))


 orgImage: m01-049.png
 File found!! .//data/dataset/forms//m01-049.png
 original image name: m01-049.png  original image shape: (3542, 2479, 3)

cropName: m01-049.png_108_[1639, 1817, 1812, 1912]_himself_.png


TypeError: predict() missing 1 required positional argument: 'test_word_label'

"""

    below code predicts PHOSC and visualizes all the image present in testCrop folder 
    to add more images into it changes are required in file 
    mnist_testYolo.txt which is in yolo prediction script
    
"""

In [51]:
"""
    gather all words
    
"""
import cv2
import os

import re
regex = re.compile('[^a-zA-Z]')

images=['m01-090.png', 'm01-110.png', 'm01-049.png', 'm01-060.png', 'm01-104.png', 'm01-095.png', 'm01-121.png', 'm01-115.png', 'm01-084.png', 'm01-079.png']
#images=['m01-084.png']

#orgImage=images[0]

testImgPath=".//data/dataset/forms//" #"/home/aniketag/Documents/phd/yolov5/data/datasets/forms/m01-084.png"
#testImgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//" 

testCropPath="./data//crops//"

images=os.listdir(testCropPath)

savePath=".//data/cropPHOSCVisual//"

print(" images:",images[0] in os.listdir(testImgPath))

    
for orgImage in images:
    
    try:
        matchCount=0
        missMatch=0
        testCropPath="./data//crops//"

        if not os.path.isdir(os.path.join(testCropPath,orgImage)):
            continue

        test_word_label,allWords=getPhoscFromText(testCropPath,orgImage)

        testCropPath=testCropPath+orgImage+"//"

        if os.path.isfile(savePath+orgImage+"_singleDict_.png"):
            continue

        #print(" imageName:",orgImage," No of unique words:",len(allWords)," is file:",os.path.isfile(testImgPath+orgImage))

        image=cv2.imread(testImgPath+orgImage)
        #print(" original image shape:",image.shape)

        df=pd.DataFrame(index=range(200),columns=["image_name","crop_name","original_word","predicted_word"])   


        for indx,cropImgName in enumerate(os.listdir(testCropPath)):

            #print(" cropImgName:",cropImgName)

            #temp=pd.DataFrame(columns=["image_name","crop_name","original_word","predicted_word"]

            word=cropImgName.split("]_")[1]
            word=word.split("_.png")[0]

            cor=cropImgName.split("[")[1]
            cor=cor.split("]")[0]
            cor="["+cor+"]"
            cor=eval(cor)

            x1,y1,x2,y2=int(cor[0]),int(cor[1]),int(cor[2]),int(cor[3])

            #imgName=imgName.split("_")[0]

            predWord,score=predict(testCropPath+cropImgName,test_word_label)

            #print(" word:",word," ",predWord," score:",score," coordinate:",x1,y1,x2,y2,)
            #print("")

            df.loc[indx,"image_name"]=orgImage
            df.loc[indx,"crop_name"]=cropImgName
            df.loc[indx,"original_word"]=word
            df.loc[indx,"predicted_word"]=predWord        

            if word==predWord:
                #cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,255,0), 3)
                #cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                matchCount+=1
            else:

                cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,0,255), 3)
                cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                missMatch+=1


        print("match",matchCount)
        print("miss match:",missMatch)
        print(" Accuracy:",matchCount/(matchCount+missMatch))

        cv2.putText(image,str([matchCount,missMatch,matchCount/(matchCount+missMatch)]), (15,100), cv2.FONT_HERSHEY_SIMPLEX, 3, 255)
        cv2.imwrite(savePath+orgImage+"_singleDictResize_.png",image)
        #df.to_csv(savePath+orgImage+"_singleDict_.csv")

        #input("check!!!")
    except Exception as e:
        pass
        

 images: False


/tmp/ipykernel_7489/215380575.py:19: RuntimeWarning: invalid value encountered in double_scalars
  return 1000*np.dot(x,y)/(LA.norm(x)*LA.norm(y))


match 50
miss match: 32
 Accuracy: 0.6097560975609756
match 29
miss match: 54
 Accuracy: 0.3493975903614458
match 54
miss match: 16
 Accuracy: 0.7714285714285715
match 47
miss match: 49
 Accuracy: 0.4895833333333333
match 50
miss match: 39
 Accuracy: 0.5617977528089888
match 59
miss match: 34
 Accuracy: 0.6344086021505376
match 47
miss match: 82
 Accuracy: 0.3643410852713178
match 58
miss match: 36
 Accuracy: 0.6170212765957447
match 41
miss match: 31
 Accuracy: 0.5694444444444444
match 63
miss match: 32
 Accuracy: 0.6631578947368421
match 65
miss match: 42
 Accuracy: 0.6074766355140186
match 63
miss match: 47
 Accuracy: 0.5727272727272728
match 35
miss match: 72
 Accuracy: 0.32710280373831774
match 46
miss match: 29
 Accuracy: 0.6133333333333333
match 60
miss match: 44
 Accuracy: 0.5769230769230769
match 55
miss match: 64
 Accuracy: 0.46218487394957986
match 37
miss match: 53
 Accuracy: 0.4111111111111111
match 36
miss match: 46
 Accuracy: 0.43902439024390244
match 38
miss match: 58
 

"""

                    BATCH VISUALIZATION

    BELOW PART CAN BE USED TO VISUALIZE THE PERFORMANCE OF ORIGINAL CROPS,
    REQUIRED CROPS ARE TAKEN FROM PHOSCNET TRAINING DATA.
    THE CROP NAMES DOES NOT INCLUDE WORDS PRESENT INSIDE IT OR COORDINATE TO HANDLE IT
    CSV data_444_april.csv IS USED WHICH HAS REQUIRED INFORMATION

"""

"""

    1. Select test image_name from data_444*.csv and fetch all records related to that.
    
    2. From fetched records get BB and text of location IE create a new dataframe which has BB and corresponding text
    * there is difference between image name and crop name
    
    3. From PHOSCNET test data separate all crops corresponding to above recods and create a separate folder for it,
    here we have only Image name from 1,2 but not corresponding crop name so we need to derive
    it with string matching.
    
    Get the PHOSC vector space for all word labels present in speecific image
    
    4. Pass those crop to PHOSCNET get prediction and visualize it..
    
"""

In [12]:
import pandas as pd
import os
import cv2
import re

"""
    allPaths
"""

csvFile="./data/dataset/data_12_april.csv"
phoscTestImgPath="./data//phoscTest//"
allPhoscTestCrops="./data/IAM_Data/IAM_test/"
#PhoscOrgTestCrops="./crop2//"
PhoscOrgTestCrops="./cropPHOSCVisual3//"

In [30]:
"""
1,2.
    input: image name for which BB,Text records are needed
    output: dataframe with relevant information
    
"""

def fetchRecord(df,imageName):
    
    temp=df[df["image_name"]==imageName]
    #print(" No of records:",temp.shape)
    return temp


"""
3.
    crops corresponding to specific image file
    a. get list of all crops used for PHOSCNET testing
    b. filter crops necessary for specific image file
    
"""
from shutil import copy2

def filterCrop(allCropList,imageName):
    
    imgCropList=[]
    
    imageName=imageName.split(".png")[0]
    
    cropPhoscTestData="./data/cropPhoscTestData//"
    basePhoscData="./data//IAM_Data/IAM_test//"
    
    for cropName in allCropList:
        
        if imageName in cropName:
            
            imgCropList.append(cropName)

    print(" TOTAL CROPS CORRESPONDING TO IMAGE "+imageName+" ARE:",len(imgCropList))
    
    
    try:
        os.mkdir(cropPhoscTestData+imageName)
    except Exception as e:
        pass
    
    for filterCropName in imgCropList:
        copy2(basePhoscData+filterCropName,cropPhoscTestData+imageName)
        
        
    print(" transferred crop:",len(cropPhoscTestData+imageName)) 
        
    return imgCropList

#imageName="m01-125.png"
#allCropList=os.listdir(allPhoscTestCrops)
#mgCropList=filterCrop(allCropList,imageName)
#imgCropList

def getPhoscFromDF(df):
    
    """
        input: Dataframe containing all image information like text,coordinate etc.
        output: dictionary having word as key and PHOSC representation as value
        
        working: similar to getPhoscFromText()
        
    """
    
    regex = re.compile('[^a-zA-Z]')
    allWords=[]


    for indx,row in df.iterrows():

        word=row["text"]
        word= regex.sub('', word)
        allWords.append(word)

    allWords=list(set(allWords))
    test_word_label=get_comb_label(allWords)
    print(len(allWords))
    #allWords
    return test_word_label,allWords

#imageName="m01-084.png"
#temp=fetchRecord(df1,imageName)
#temp.columns
#test_word_label=getPhoscFromDF(temp)
#temp.head(5)

"""
    save crops from original coordinates
    As there is no way to map PHOSCNET test data crops to original image 
    
    below function extracts crops from original image, 
    The crops are saved in following format ImgName.png_IndexNumber_[x1,y1, x2, y2]_textString_.png
    
    The crop coordinate are from original dataset It is not predicted by YOLOV3
    
"""

def PhoscOrgCrops(df,imgName):
    
    image=cv2.imread(phoscTestImgPath+imgName)
    
    #print(image.shape)
    
    regex = re.compile('[^a-zA-Z]')
    
    try:
        os.mkdir(PhoscOrgTestCrops+imgName)
    
    except Exception as e:
        pass
    
    for indx,row in df.iterrows():
        #print(row)
        word=row["text"]
        word= regex.sub('', word)
        
        if len(word)<2:
            continue

        x1,y1,x2,y2=row["org_x1"],row["org_y1"],row["org_x2"],row["org_y2"]
        
        tempName=imgName+"_"+str(indx)+"_"+str([x1,y1,x2,y2])+"_"+word+".png"
        
        cropImage=image[y1:y2,x1:x2]
        #print(cropImage.shape)

        cv2.imwrite(PhoscOrgTestCrops+imgName+"//"+tempName,cropImage)
        


#PhoscOrgCrops(temp,imageName)



"""

    BELOW PART PROJECTS RESULTS FROM ORIGINAL CROPS..
    

"""

In [36]:
"""
    here
    gather all words
"""
import cv2
import os
import sys
import re
regex = re.compile('[^a-zA-Z]')

#images=['m01-090.png', 'm01-110.png', 'm01-049.png', 'm01-060.png', 'm01-104.png', 'm01-095.png', 'm01-121.png', 'm01-115.png', 'm01-084.png', 'm01-079.png']

#PhoscOrgTestCrops="./data//cropPhoscTestData2//"# crops_yolo3

#PhoscOrgTestCrops="./data//crops_yolo3//"# crops_nms
PhoscOrgTestCrops="./data//crops_nms2//"


availableCropsImg=os.listdir(PhoscOrgTestCrops)

images=os.listdir(phoscTestImgPath)

savePath="./data//cropPHOSCVisual3//"
testImgPath=".//data/dataset/forms//" 
print(" images:",images[0] in os.listdir(testImgPath))

allCropList=os.listdir(allPhoscTestCrops)

df1=pd.read_csv("./data/dataset/data_14_april.csv")


for imgNumber,orgImage in enumerate(images):
    
    try:
        #print("")
        
        """
        if os.path.isfile(savePath+orgImage+"_originalCrops_.png"):
            continue
        """ 
        """
        if os.path.isfile(savePath+orgImage+"_originalTest_.png"):
            continue
        """
        if os.path.isfile(savePath+orgImage+"_yolo3Crops_.png"):
            continue

            
        if not orgImage in availableCropsImg:
            continue
            
        matchCount=0
        missMatch=0
        
        #PhoscOrgTestCrops="./data//cropPhoscTestData3//"
        #PhoscOrgTestCrops="./data//crops_yolo3//"# 
        PhoscOrgTestCrops="./data//crops_nms2//"
        #PhoscOrgTestCrops="./data//crop2//"

        
        temp=fetchRecord(df1,orgImage) # get relevant records

        #PhoscOrgCrops(temp,orgImage) # create crops
        
        if 0:
            filterCrop(allCropList,orgImage)
        
        #test_word_label,allWords=getPhoscFromText(PhoscOrgTestCrops,orgImage) # get attribute space

        test_word_label,allWords=getPhoscFromDF(temp)
        
        if os.path.isdir(PhoscOrgTestCrops+orgImage+"//"):
            PhoscOrgTestCrops=PhoscOrgTestCrops+orgImage+"//"
        else:
            PhoscOrgTestCrops=PhoscOrgTestCrops+orgImage[:-4]+"//"            
        
        noTestCrops=len(os.listdir(PhoscOrgTestCrops))
        
        #print(" imageName:",orgImage," No of unique words:",len(allWords)," is file:",os.path.isfile(testImgPath+orgImage))

        print(" imageName:",orgImage," No of unique words:",len(test_word_label.keys())," # crops:",noTestCrops," is file:",os.path.isfile(phoscTestImgPath+orgImage))


        image=cv2.imread(phoscTestImgPath+orgImage)
        #print(" original image shape:",image.shape)
    
        
        df=pd.DataFrame(index=range(200),columns=["image_name","crop_name","original_word","predicted_word"])   

        for indx,cropImgName in enumerate(os.listdir(PhoscOrgTestCrops)):

            #print(" cropImgName:",cropImgName)

            #temp=pd.DataFrame(columns=["image_name","crop_name","original_word","predicted_word"]
            
            
            try:
                word=cropImgName.split("]_")[1]
                word=word.split(".png")[0]

                cor=cropImgName.split("[")[1]
                cor=cor.split("]")[0]
                cor="["+cor+"]"
                cor=eval(cor)
                
            except Exception as e:
                tempCropName=cropImgName[:-4]
                tempRow=df1[df1["cropName"]==tempCropName]
                #print("")
                #print("\n tempRow \n ",tempRow)
                #print("")

                word=tempRow.text.values[0]
                cor=[tempRow.org_x1.values,tempRow.org_y1.values,tempRow.org_x2.values,tempRow.org_y2.values]
                #input(" check row")
                
                #print(" word=",word)
                #print(" cor=",cor)
                #print(" cropImgName:",cropImgName)
                
                
                
            x1,y1,x2,y2=int(cor[0]),int(cor[1]),int(cor[2]),int(cor[3])
            
            #imgName=imgName.split("_")[0]
            
            try:
                predWord,score=predict(PhoscOrgTestCrops+cropImgName,test_word_label)
            except Exception as e:
                
                predWord=""
                
            #print("\n word:",word,"-->",predWord," score:",score," coordinate:",x1,y1,x2,y2," \n cropImgName:",cropImgName)
            #predWord=predWord[:-3]
            
            try:
                df.loc[indx,"image_name"]=orgImage
                df.loc[indx,"crop_name"]=cropImgName
                df.loc[indx,"original_word"]=word
                df.loc[indx,"predicted_word"]=predWord        
            except Exception as e:
                exc_type, exc_obj, exc_tb = sys.exc_info()
                fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                print(" line number:", exc_tb.tb_lineno, " e:",e)
            
            if word[:-1]==predWord:
                #cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,255,0), 4)
                #cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                matchCount+=1
            else:

                cv2.putText(image,predWord, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,0,255), 3)
                cv2.putText(image,word, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                missMatch+=1
                #cv2.imwrite(savePath+orgImage+"_originalTest_.png",image)

                #input(" missmatch!!!")


        print("match",matchCount)
        print("miss match:",missMatch)
        print(" Accuracy:",matchCount/(matchCount+missMatch))

        cv2.putText(image,str([matchCount,missMatch,matchCount/(matchCount+missMatch)]), (15,100), cv2.FONT_HERSHEY_SIMPLEX, 3, 255)
        #cv2.imwrite(savePath+orgImage+"_originalTest_.png",image)
        #df.to_csv(savePath+orgImage+"_originalTest_.csv")
        
        cv2.imwrite(savePath+orgImage+"_yolo33Crops_.png",image)
        #df.to_csv(savePath+orgImage+"_yolo2C33333rops_.csv")

        
            
    except Exception as e1:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(" line number:", exc_tb.tb_lineno, " e:",e1)
        
    #input("check!!!")
    

 images: True
48
 imageName: m01-131.png  No of unique words: 48  # crops: 62  is file: True


/tmp/ipykernel_35523/215380575.py:19: RuntimeWarning: invalid value encountered in double_scalars
  return 1000*np.dot(x,y)/(LA.norm(x)*LA.norm(y))


match 32
miss match: 30
 Accuracy: 0.5161290322580645
45
 imageName: m02-075.png  No of unique words: 45  # crops: 58  is file: True
match 19
miss match: 39
 Accuracy: 0.3275862068965517
43
 imageName: m02-112.png  No of unique words: 43  # crops: 50  is file: True
match 36
miss match: 14
 Accuracy: 0.72
52
 imageName: m01-136.png  No of unique words: 52  # crops: 69  is file: True
match 28
miss match: 41
 Accuracy: 0.4057971014492754
51
 imageName: m03-062.png  No of unique words: 51  # crops: 64  is file: True
match 32
miss match: 32
 Accuracy: 0.5
50
 imageName: m01-060.png  No of unique words: 50  # crops: 66  is file: True
match 39
miss match: 27
 Accuracy: 0.5909090909090909
73
 imageName: m02-069.png  No of unique words: 73  # crops: 94  is file: True
match 22
miss match: 72
 Accuracy: 0.23404255319148937
53
 imageName: m01-115.png  No of unique words: 53  # crops: 71  is file: True
match 44
miss match: 27
 Accuracy: 0.6197183098591549
42
 imageName: m01-160.png  No of unique wo